In [2]:
import pandas as pd                                                                                                                                                                                                                                                             
from sklearn.svm import SVC                                                                                                                                                                                                                                                     
import joblib         
import numpy as np
        
# TO DO:
# Feature comparison (between my version of SV2 and Danny's version)
# Create new classifiers and run on my version of SV2

# Compare my features with Danny's.
# Check each copy number value and make sure that the correspondence is the same for mine and Danny's.
# Train classifiers.
# Run on HG002 features.
    
def open_train(filepath):                                                                                                                                                                                                                                                       
    df = pd.read_csv(filepath, sep = "\t")                                                                                                                                                                                                                                      
    return df[df["covr"] < 5.0] # Do not train on outliers                                                                                                                                                                                                                      


# Generate weights for biallelic SVs
# Genotypes must be ordered as [REF, HET, HOM]
def make_biallelic_weights(coverage, genotype, heterozygous_expected_coverage, homozygous_expected_coverage):
    buffer = 0.01
    expected_coverage = 1.0
    if genotype == 1: expected_coverage = heterozygous_expected_coverage
    elif genotype == 0: expected_coverage = homozygous_expected_coverage
    return 1.0/(abs(expected_coverage - coverage) + buffer)

# For male sex chromosomes: Danny creates a weight for each sample as a function of the sample's coverage and genotype
# Genotypes must be ordered as [REF, ALT]
def make_allele_weights(coverage, genotype, alternative_allele_expected_coverage):
    buffer = 0.01
    expected_coverage = 1.0
    # Alternate genotype
    if genotype == 0: expected_coverage = alternative_allele_expected_coverage
    return 1.0/(abs(expected_coverage - coverage) + buffer)
    
def make_snv_weights(coverage, genotype, heterozygous_expected_coverage, homozygous_expected_coverage, heterozygous_allele_ratio, median_heterozygous_ratio_copy_number_2, median_heterozygous_ratio_copy_number_3, median_heterozygous_ratio_copy_number_4):
    buffer = 0.01
    if genotype == 2:
        return 1.0 / np.sqrt(( ((1 - coverage)**2) + ((median_heterozygous_ratio_copy_number_2 - heterozygous_allele_ratio)**2) + buffer ) )
    elif genotype == 1:
        return 1.0 / np.sqrt(( ((heterozygous_expected_coverage - coverage)**2) + ((median_heterozygous_ratio_copy_number_3 - heterozygous_allele_ratio)**2) + buffer ) )
    elif genotype == 0:
        return max(1.0 / np.sqrt(( ((homozygous_expected_coverage - coverage)**2) + ((median_heterozygous_ratio_copy_number_2 - heterozygous_allele_ratio)**2) + buffer ) ),
                   1.0 / np.sqrt(( ((homozygous_expected_coverage - coverage)**2) + ((median_heterozygous_ratio_copy_number_4 - heterozygous_allele_ratio)**2) + buffer ) ) )
                   
        
MEM = 12000                                                                                                                                                                                                                                                                     
pd.set_option('display.max_rows', 30)

In [ ]:
# Deletion > 1000 bp classifier     
df_highcov_del_gt1kb = open_train("data/training_set_files/1kgp_highcov_del_gt1kb.txt")[["covr", "dpe_rat", "sr_rat", "copy_number"]]    
sample_weights = df_highcov_del_gt1kb.apply(lambda sample: make_biallelic_weights(sample["covr"], sample["copy_number"], 0.5, 0.0), axis = 1)
clf_del_gt1kb = SVC(kernel = "rbf" , probability = True, random_state = 42, C = 0.01, gamma = 10, class_weight = "balanced", cache_size = MEM)
clf_del_gt1kb.fit(df_highcov_del_gt1kb.values[:, 0:-1], df_highcov_del_gt1kb.values[:, -1], sample_weight = sample_weights)                                                          
joblib.dump(clf_del_gt1kb, "new_classifiers/clf_del_gt1kb.pkl")  

In [76]:
# Deletion <= 1000 Classifier             
df_highcov_del_lt1kb = open_train("data/training_set_files/1kgp_highcov_del_lt1kb.txt")[["covr", "dpe_rat", "sr_rat", "copy_number"]]
sample_weights = df_highcov_del_lt1kb.apply(lambda sample: make_biallelic_weights(sample["covr"], sample["copy_number"], 0.5, 0.0), axis = 1)
clf_del_lt1kb = SVC(kernel = "rbf" ,probability = True, random_state = 42, C = 1000, gamma = 1, class_weight = "balanced", cache_size = MEM)
clf_del_lt1kb.fit(df_highcov_del_lt1kb.values[:, 0:-1], df_highcov_del_lt1kb.values[:, -1], sample_weight = sample_weights)
joblib.dump(clf_del_lt1kb, "clf_del_lt1kb.pkl")   

<ipython-input-76-0217bdc51f4b>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_highcov_del_lt1kb.sort_values(by = ["chr", "start", "end", "id"])[df_highcov_del_lt1kb["copy_number"] == 0]


,chr,start,end,type,size,id,coverage,covr,dpe_rat,sr_rat,HET_ratio,normHET_ratio,HET_SNPs,copy_number,sample_weights
340,chr1,1142714,1143140,DEL,427,NA18525,0.0,0.0,0.005,0.214,NaN,NaN,0,0,100.0
377,chr1,11682873,11683189,DEL,317,HG00268,0.0,0.0,0.007,0.160,NaN,NaN,0,0,100.0
378,chr1,11682873,11683189,DEL,317,HG00419,0.0,0.0,0.000,0.169,NaN,NaN,0,0,100.0
380,chr1,11682873,11683189,DEL,317,HG01051,0.0,0.0,0.000,0.164,NaN,NaN,0,0,100.0
381,chr1,11682873,11683189,DEL,317,HG01112,0.0,0.0,0.007,0.182,NaN,NaN,0,0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62211,chrX,154790161,154790501,DEL,341,HG01595,0.0,0.0,0.000,0.000,NaN,NaN,0,0,100.0
62216,chrX,154790161,154790501,DEL,341,NA12878,0.0,0.0,0.000,0.000,NaN,NaN,0,0,100.0
62217,chrX,154790161,154790501,DEL,341,NA18525,0.0,0.0,0.000,0.000,NaN,NaN,0,0,100.0
62218,chrX,154790161,154790501,DEL,341,NA18939,0.0,0.0,0.000,0.000,NaN,NaN,0,0,100.0


In [80]:
# Deletion male sex chromosome classifier
df_del_malesexchrom = open_train("data/training_set_files/1kgp_highcov_del_malesexchrom.txt")[["covr", "dpe_rat", "sr_rat", "copy_number"]]            
sample_weights = df_del_malesexchrom.apply(lambda sample: make_allele_weights(sample["covr"], sample["copy_number"], 0.0), axis = 1)
clf_del_malesexchrom = SVC(kernel = "rbf", probability = True, random_state = 42, C = 1, gamma = 1, class_weight = "balanced", cache_size = MEM)           
clf_del_malesexchrom.fit(df_del_malesexchrom.values[:, 0:-1], df_del_malesexchrom.values[:, -1], sample_weight = sample_weights)
joblib.dump(clf_del_malesexchrom, "clf_del_malesexchrom.pkl")

<ipython-input-80-0b62769a99b9>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_del_malesexchrom.sort_values(by = ["chr", "start", "end", "id"])[df_del_malesexchrom["copy_number"] == 0]


,chr,start,end,type,size,id,covr,dpe,sr,conc,dpe_rat,sr_rat,geno,CS,copy_number,sample_weights
3304,chrX,4233776,4234935,DEL,1160,NA19239,0.050,59,31,124,0.476,0.250,0,DEL_union,0,16.666667
3,chrX,5055345,5057501,DEL,2157,HG00096,0.043,24,15,125,0.192,0.120,0,DEL_union,0,18.867925
370,chrX,5055345,5057501,DEL,2157,HG01051,0.047,40,25,133,0.301,0.188,0,DEL_union,0,17.543860
737,chrX,5055345,5057501,DEL,2157,HG01112,0.051,23,14,130,0.177,0.108,0,DEL_union,0,16.393443
1104,chrX,5055345,5057501,DEL,2157,HG01500,0.070,34,18,138,0.246,0.130,0,DEL_union,0,12.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4035,chrX,154798201,154803718,DEL,5518,NA20845,0.566,0,0,56,0.000,0.000,0,DEL_union,0,1.736111
366,chrX,154918469,154921719,DEL,3251,HG00096,0.000,30,7,169,0.178,0.041,0,DEL_union,0,100.000000
1100,chrX,154918469,154921719,DEL,3251,HG01112,0.000,23,1,126,0.183,0.008,0,DEL_union,0,100.000000
1834,chrX,154918469,154921719,DEL,3251,HG01565,0.000,14,1,97,0.144,0.010,0,DEL_union,0,100.000000


In [83]:
# Duplication breakpoint classifier
# df_dup_breakpoint["copy_number"].unique() -> array([2, 3, 4])
df_dup_breakpoint = open_train("data/training_set_files/1kgp_lowcov_dup_breakpoint.txt")[["covr", "dpe_rat", "sr_rat", "copy_number"]]
# Danny defines a function called "dup_convert" (for duplications) where he converts "copy_number" 3 to 1, and 4 to 0 (and default is 2)
df_dup_breakpoint["copy_number"] = df_dup_breakpoint["copy_number"].mask(df_dup_breakpoint["copy_number"] == 3, 1)
df_dup_breakpoint["copy_number"] = df_dup_breakpoint["copy_number"].mask(df_dup_breakpoint["copy_number"] == 4, 0)
class_weights = {0: 2, 1: 0.5, 2: 1}    
sample_weights = df_dup_breakpoint.apply(lambda sample: make_biallelic_weights(sample["covr"], sample["copy_number"], 1.5, 2.0), axis = 1)
# clf_dup_breakpoint = SVC(kernel = "rbf", probability = True, random_state = 42, C = 1, gamma = 10, class_weight = class_weights, cache_size = MEM)
# clf_dup_breakpoint.fit(df_dup_breakpoint.values[:, 0:-1], df_dup_breakpoint.values[:, -1], sample_weight = sample_weights)                                            
# joblib.dump(clf_dup_breakpoint, "clf_dup_breakpoint.pkl") 

<ipython-input-83-92d0e93fdfaf>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_dup_breakpoint.sort_values(by = ["chr", "start", "end", "id"])[df_dup_breakpoint["copy_number"] == 0]


,chr,start,end,type,size,id,covrNo,covr,dpe_rat,sr_rat,copy_number,sample_weights
4852,chr1,2466216,2470030,DUP,3815,HG00097,0.62,0.775,0.015,0.0,0,0.809717
4853,chr1,2466216,2470030,DUP,3815,HG00099,0.55,0.688,0.009,0.0,0,0.756430
4854,chr1,2466216,2470030,DUP,3815,HG00128,1.02,1.275,0.000,0.0,0,1.360544
4855,chr1,2466216,2470030,DUP,3815,HG00130,0.79,0.988,0.018,0.0,0,0.978474
4856,chr1,2466216,2470030,DUP,3815,HG00234,0.86,1.075,0.000,0.0,0,1.069519
...,...,...,...,...,...,...,...,...,...,...,...,...
118382,chrX,154891374,155047898,DUP,156525,HG03267,1.55,1.462,0.000,0.0,0,1.824818
118414,chrX,155070527,155155923,DUP,85397,NA18916,1.59,1.500,0.000,0.0,0,1.960784
118423,chrX,155173908,155221605,DUP,47698,NA18916,1.55,1.462,0.000,0.0,0,1.824818
118426,chrX,155429032,155453911,DUP,24880,HG00118,1.43,1.349,0.000,0.0,0,1.512859


In [86]:
# Duplication HAR classifier                                                                                                            
df_dup_har = open_train("data/training_set_files/1kgp_highcov_dup_har.txt")[["covr", "HET_ratio", "copy_number"]]
df_dup_har["copy_number"] = df_dup_har["copy_number"].mask(df_dup_har["copy_number"] == 3, 1)
df_dup_har["copy_number"] = df_dup_har["copy_number"].mask(df_dup_har["copy_number"] == 4, 0)
median_heterozygous_ratio_copy_number_2 = np.median(df_dup_har[df_dup_har["copy_number"] == 2]["HET_ratio"])
median_heterozygous_ratio_copy_number_3 = np.median(df_dup_har[df_dup_har["copy_number"] == 1]["HET_ratio"])
median_heterozygous_ratio_copy_number_4 = median_heterozygous_ratio_copy_number_2/2.0
sample_weights = df_dup_har.apply(lambda sample: make_snv_weights(sample["covr"], sample["copy_number"], 1.5, 2.0, sample["HET_ratio"], median_heterozygous_ratio_copy_number_2, median_heterozygous_ratio_copy_number_3, median_heterozygous_ratio_copy_number_4), axis = 1)

# clf_dup_har = SVC(kernel = "rbf", probability = True, random_state = 42, C = 0.01, gamma = 5000, class_weight = "balanced", cache_size
# clf_dup_har.fit(df_dup_har.values[:, 0:-1], df_dup_har.values[:, -1])                                                                 
# joblib.dump(clf_dup_har, "clf_dup_har.pkl")    

<ipython-input-86-baafe09fbb33>:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_dup_har.sort_values(by = ["chr", "start", "end", "id"])[df_dup_har["copy_number"] == 0]


,chr,start,end,type,size,id,coverage,covr_GC,dpe_rat,sr_rat,SNP_coverage,SNPs,HET_ratio,HET_SNPs,copy_number,covr,sample_weights
517,chr1,85980461,86006144,DUP,25684,HG00096,2.183,2.059,0.0,0.0,2.227,303,0.800,283,0,2.14,5.682460
518,chr1,85980461,86006144,DUP,25684,HG00268,2.096,1.977,0.0,0.0,2.149,305,0.820,289,0,2.06,7.703941
519,chr1,85980461,86006144,DUP,25684,HG00419,2.115,1.995,0.0,0.0,2.158,309,0.822,292,0,2.08,7.092199
520,chr1,85980461,86006144,DUP,25684,HG00759,2.172,2.049,0.0,0.0,2.297,318,0.812,299,0,2.17,4.920619
521,chr1,85980461,86006144,DUP,25684,HG01051,2.244,2.117,0.0,0.0,2.281,306,0.789,289,0,2.20,4.442207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8768,chrX,75301385,75313459,DUP,12075,NA19238,2.817,2.515,0.0,0.0,2.103,30,0.369,11,0,2.31,3.067770
8530,chrX,119029253,119058207,DUP,28955,HG02922,2.076,2.076,0.0,0.0,1.467,49,0.396,37,0,1.77,3.980614
8531,chrX,119029253,119058207,DUP,28955,HG03052,2.037,2.037,0.0,0.0,1.432,47,0.412,31,0,1.73,3.453819
8562,chrX,119036694,119055824,DUP,19131,HG02922,2.132,2.132,0.0,0.0,1.533,36,0.388,28,0,1.83,5.067450


In [4]:
# Duplication male sex chromosome classifier
# df_dup_malesexchrom["copy_number"].unique() -> array([1, 3])
df_dup_malesexchrom = open_train("data/training_set_files/1kgp_lowcov_dup_malesexchrom.txt")[["covr", "dpe_rat", "sr_rat", "copy_number"]]                                                                                                                                      
# Danny defines a function called "dup_convert_msc" (for duplications on the male sex chromosomes) where he changes "copy_number" to 0 if it's greater than 1
df_dup_malesexchrom["copy_number"] = df_dup_malesexchrom["copy_number"].mask(df_dup_malesexchrom["copy_number"] > 1, 0)
class_weights = {0: 1, 1: 1}    
sample_weights = df_dup_malesexchrom.apply(lambda sample: make_allele_weights(sample["covr"], sample["copy_number"], 2.0), axis = 1)
# clf_dup_malesexchrom = SVC(kernel = "rbf", probability = True, random_state = 42, C = 1000, gamma = 0.01, class_weight = class_weights, cache_size = MEM)                                
# clf_dup_malesexchrom.fit(df_dup_malesexchrom.values[:, 0:-1], df_dup_malesexchrom.values[:, -1], sample_weight = sample_weights)
# joblib.dump(clf_dup_malesexchrom, "clf_dup_malesexchrom.pkl")